Selection Top 20 Features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# load data
df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
df = pd.concat([df1, df2], ignore_index=True)

# correct erroneous values
df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) & (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
condition = df["avg_available_spots"] > df["TotalSpaces"]
df.loc[condition, "avg_available_spots"] = df.loc[condition, "TotalSpaces"]

# Add day_off and wind direction sin/cos
df['datetime'] = pd.to_datetime(df['datetime'])
df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
df['day_off'] = df['day_off'].astype(int)

# wind direction sin/cos transformation
df['wind_direction_10m_sin'] = np.sin(np.deg2rad(df['wind_direction_10m']))
df['wind_direction_10m_cos'] = np.cos(np.deg2rad(df['wind_direction_10m']))
df['wind_direction_80m_sin'] = np.sin(np.deg2rad(df['wind_direction_80m']))
df['wind_direction_80m_cos'] = np.cos(np.deg2rad(df['wind_direction_80m']))
df['wind_direction_120m_sin'] = np.sin(np.deg2rad(df['wind_direction_120m']))
df['wind_direction_120m_cos'] = np.cos(np.deg2rad(df['wind_direction_120m']))
df.drop(columns=["wind_direction_10m","wind_direction_120m","wind_direction_80m","holiday_name","is_national_holiday"], inplace=True)

# time split
df = df.dropna(subset=['avg_available_spots'])
train_start = pd.to_datetime("2024-01-23")
train_end   = pd.to_datetime("2025-01-22")
valid_start = pd.to_datetime("2025-01-23")
valid_end   = pd.to_datetime("2025-03-22")
test_start  = pd.to_datetime("2025-03-23")
test_end    = pd.to_datetime("2025-05-09")


df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
df_test  = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

# remove unnecessary columns
drop_cols = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID']
for dataset in [df_train, df_valid, df_test]:
    for col in drop_cols:
        if col in dataset.columns:
            dataset.drop(columns=col, inplace=True)

# create X and y, including cyclical features and one-hot encoding
def prepare_xy(df_part):
    y = df_part['avg_available_spots']
    X = df_part.drop(columns=['avg_available_spots'])

    X['sin_day'] = np.sin(2 * np.pi * X['day_val'] / 31)
    X['cos_day'] = np.cos(2 * np.pi * X['day_val'] / 31)
    X['sin_halfhour'] = np.sin(2 * np.pi * X['half_hour_interval'] / 48)
    X['cos_halfhour'] = np.cos(2 * np.pi * X['half_hour_interval'] / 48)
    X.drop(columns=['day_val', 'half_hour_interval'], inplace=True)

    cat_cols = ['month_val', 'weekday_val'] + X.select_dtypes(include=['object']).columns.tolist()
    cat_cols_exist = [col for col in cat_cols if col in X.columns]
    X = pd.get_dummies(X, columns=cat_cols_exist, drop_first=False)
    return X, y

X_train, y_train = prepare_xy(df_train)
X_valid, y_valid = prepare_xy(df_valid)
X_test,  y_test  = prepare_xy(df_test)

X_valid = X_valid.reindex(columns=X_train.columns, fill_value=0)
X_test  = X_test.reindex(columns=X_train.columns, fill_value=0)

# grid search
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
model = XGBRegressor(random_state=42)
grid = GridSearchCV(model, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_

# evaluate the test set
for name, X_part, y_part in [('\u6e2c\u8a66\u96c6', X_test, y_test)]:
    y_pred = best_model.predict(X_part)
    print(f"\n\ud83d\udcca {name} \u8a55\u4f30：")
    print(f"MAE: {mean_absolute_error(y_part, y_pred):.2f}")
    print(f"RMSE: {mean_squared_error(y_part, y_pred)**0.5:.2f}")
    print(f"R2 Score: {r2_score(y_part, y_pred):.4f}")

# feature importance analysis
booster = best_model.get_booster()
importance_types = ['gain', 'weight', 'cover']
importance_df = pd.DataFrame({'feature': X_train.columns})

for imp_type in importance_types:
    scores = booster.get_score(importance_type=imp_type)
    importance_df[imp_type] = importance_df['feature'].map(scores).fillna(0)

importance_df_sorted = importance_df.sort_values(by='gain', ascending=False)

print("\n\ud83d\udccb Top 22 \u7279\u5fb5 (by gain):")
print(importance_df_sorted.head(20).to_string(index=False))

plt.figure(figsize=(12, 6))
sns.barplot(data=importance_df_sorted.head(20), x='gain', y='feature')
plt.title("Top 20 \u7279\u5fb5\u91cd\u8981\u6027 (by Gain)")
plt.xlabel("Gain")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


Regression with All Features

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Binarizer

# ========= data loading and preprocessing =========
df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
df = pd.concat([df1, df2], ignore_index=True)

# correct erroneous values
df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) & (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
df.loc[df["avg_available_spots"] > df["TotalSpaces"], "avg_available_spots"] = df["TotalSpaces"]

# Add `day_off` and wind direction `sin`/`cos`.
df['datetime'] = pd.to_datetime(df['datetime'])
df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
df['day_off'] = df['day_off'].astype(int)
for height in [10, 80, 120]:
    df[f'wind_direction_{height}m_sin'] = np.sin(np.deg2rad(df[f'wind_direction_{height}m']))
    df[f'wind_direction_{height}m_cos'] = np.cos(np.deg2rad(df[f'wind_direction_{height}m']))
df.drop(columns=[
    "wind_direction_10m", "wind_direction_80m", "wind_direction_120m",
    "holiday_name", "is_national_holiday"
], inplace=True)

# time split
df = df.dropna(subset=['avg_available_spots'])
train_start = pd.to_datetime("2024-01-23")
train_end = pd.to_datetime("2025-01-22")
valid_start = pd.to_datetime("2025-01-23")
valid_end = pd.to_datetime("2025-03-22")
test_start = pd.to_datetime("2025-03-23")
test_end = pd.to_datetime("2025-05-09")

df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
df_test = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

drop_cols = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID']
for dataset in [df_train, df_valid, df_test]:
    for col in drop_cols:
        if col in dataset.columns:
            dataset.drop(columns=col, inplace=True)

# ========= feature engineering function =========
def prepare_xy(df_part):
    y = df_part['avg_available_spots']
    X = df_part.drop(columns=['avg_available_spots'])

    # cyclical features
    X['sin_day'] = np.sin(2 * np.pi * X['day_val'] / 31)
    X['cos_day'] = np.cos(2 * np.pi * X['day_val'] / 31)
    X['sin_halfhour'] = np.sin(2 * np.pi * X['half_hour_interval'] / 48)
    X['cos_halfhour'] = np.cos(2 * np.pi * X['half_hour_interval'] / 48)
    X.drop(columns=['day_val', 'half_hour_interval'], inplace=True)

    # One-Hot encoding
    cat_cols = ['month_val', 'weekday_val'] + X.select_dtypes(include=['object']).columns.tolist()
    cat_cols_exist = [col for col in cat_cols if col in X.columns]
    X = pd.get_dummies(X, columns=cat_cols_exist, drop_first=False)
    return X, y

X_train, y_train = prepare_xy(df_train)
X_valid, y_valid = prepare_xy(df_valid)
X_test, y_test = prepare_xy(df_test)

# ensure column consistency
X_valid = X_valid.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# ========= model training and frid search =========
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
model = XGBRegressor(random_state=42)
grid = GridSearchCV(model, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_

# ========= prediction and metrics =========
def adjusted_r2(r2, n, k):
    return 1 - (1 - r2) * ((n - 1) / (n - k - 1))

def eval_all(X, y, dataset_name):
    y_pred = best_model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    adj_r2 = adjusted_r2(r2, len(y), X.shape[1])

    # classification task: whether there is an available space
    y_true_bin = (y > 0).astype(int)
    best_threshold = 0.5
    best_f1 = 0
    for t in np.arange(0.1, 0.91, 0.01):
        pred_bin = (y_pred > t).astype(int)
        f1 = f1_score(y_true_bin, pred_bin)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t

    pred_bin_best = (y_pred > best_threshold).astype(int)
    acc = accuracy_score(y_true_bin, pred_bin_best)
    prec = precision_score(y_true_bin, pred_bin_best)
    rec = recall_score(y_true_bin, pred_bin_best)
    f1 = f1_score(y_true_bin, pred_bin_best)
    roc = roc_auc_score(y_true_bin, y_pred)
    prc = average_precision_score(y_true_bin, y_pred)

    print(f"\n📊 [{dataset_name}]")
    print(f"MSE: {mse:.2f}")
    print(f"Adjusted R²: {adj_r2:.4f}")
    # print(f"R²: {r2:.4f}")
    # print(f"Accuracy: {acc:.4f}")
    # print(f"Precision: {prec:.4f}")
    # print(f"Recall: {rec:.4f}")
    # print(f"F1 Score: {f1:.4f}")
    # print(f"ROC-AUC: {roc:.4f}")
    # print(f"PRC-AUC: {prc:.4f}")
    # print(f"Best Threshold: {best_threshold:.3f}")

# evalation
# eval_all(X_train, y_train, "Train")
# eval_all(X_valid, y_valid, "Valid")
eval_all(X_test, y_test, "Test")


Fitting 3 folds for each of 8 candidates, totalling 24 fits

📊 [Test]
MSE: 103.86
Adjusted R²: 0.9720


Regression with Features of the Four Votes

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# feature list
selected_features_4_votes = [
    'TotalSpaces', 'temperature_2m', 'hour_val', 'day_off',
    'wind_speed_10m', 'wind_speed_80m', 'wind_speed_120m',
    'wind_direction_10m_sin', 'wind_direction_10m_cos',
    'wind_direction_80m_sin', 'wind_direction_80m_cos',
    'wind_direction_120m_sin', 'wind_direction_120m_cos',
    'weather_code', 'sin_halfhour', 'cos_halfhour',
    'sin_day', 'cos_day'
]

# data loading and preprocessing
df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
df = pd.concat([df1, df2], ignore_index=True)

# correct erroneous values
df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) & (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
df.loc[df["avg_available_spots"] > df["TotalSpaces"], "avg_available_spots"] = df["TotalSpaces"]

df['datetime'] = pd.to_datetime(df['datetime'])
df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
df['day_off'] = df['day_off'].astype(int)
for height in [10, 80, 120]:
    df[f'wind_direction_{height}m_sin'] = np.sin(np.deg2rad(df[f'wind_direction_{height}m']))
    df[f'wind_direction_{height}m_cos'] = np.cos(np.deg2rad(df[f'wind_direction_{height}m']))
df.drop(columns=[
    "wind_direction_10m", "wind_direction_80m", "wind_direction_120m",
    "holiday_name", "is_national_holiday"
], inplace=True)

# time split
df = df.dropna(subset=['avg_available_spots'])
train_start = pd.to_datetime("2024-01-23")
train_end = pd.to_datetime("2025-01-22")
valid_start = pd.to_datetime("2025-01-23")
valid_end = pd.to_datetime("2025-03-22")
test_start = pd.to_datetime("2025-03-23")
test_end = pd.to_datetime("2025-05-09")

df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
df_test = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

drop_cols = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID']
for dataset in [df_train, df_valid, df_test]:
    for col in drop_cols:
        if col in dataset.columns:
            dataset.drop(columns=col, inplace=True)

# feature engineering + cyclical features + One-Hot encoding
def prepare_xy_vote(df_part):
    y = df_part['avg_available_spots']
    X = df_part.drop(columns=['avg_available_spots'])

    X['sin_day'] = np.sin(2 * np.pi * X['day_val'] / 31)
    X['cos_day'] = np.cos(2 * np.pi * X['day_val'] / 31)
    X['sin_halfhour'] = np.sin(2 * np.pi * X['half_hour_interval'] / 48)
    X['cos_halfhour'] = np.cos(2 * np.pi * X['half_hour_interval'] / 48)
    X.drop(columns=['day_val', 'half_hour_interval'], inplace=True)

    cat_cols = ['month_val', 'weekday_val'] + X.select_dtypes(include=['object']).columns.tolist()
    cat_cols_exist = [col for col in cat_cols if col in X.columns]
    X = pd.get_dummies(X, columns=cat_cols_exist, drop_first=False)

    X = X.reindex(columns=selected_features_4_votes, fill_value=0)
    return X, y

X_train, y_train = prepare_xy_vote(df_train)
X_valid, y_valid = prepare_xy_vote(df_valid)
X_test, y_test = prepare_xy_vote(df_test)

# model training
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
model = XGBRegressor(random_state=42)
grid = GridSearchCV(model, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_

# prediction and evaluation
def adjusted_r2(r2, n, k):
    return 1 - (1 - r2) * ((n - 1) / (n - k - 1))

def eval_regression(X, y, dataset_name):
    y_pred = best_model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    adj_r2 = adjusted_r2(r2, len(y), X.shape[1])
    print(f"\n📊 [{dataset_name} - 投票四票]")
    print(f"MSE: {mse:.2f}")
    print(f"Adjusted R²: {adj_r2:.4f}")

# evaluation
# eval_regression(X_train, y_train, "Train")
# eval_regression(X_valid, y_valid, "Valid")
eval_regression(X_test, y_test, "Test")


/var/folders/r9/vznk0t21541cq588wv51gt6m0000gn/T/ipykernel_3636/147346167.py:19: DtypeWarning: Columns (47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
/var/folders/r9/vznk0t21541cq588wv51gt6m0000gn/T/ipykernel_3636/147346167.py:20: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")


Fitting 3 folds for each of 8 candidates, totalling 24 fits

📊 [Test - 投票四票]
MSE: 129.21
Adjusted R²: 0.9652


Regression with Features of the Three Votes

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# feature list
selected_features_3_votes = [
    'TotalSpaces', 'temperature_2m', 'hour_val', 'day_off',
    'wind_speed_10m', 'wind_speed_80m',
    'wind_direction_10m_sin', 'wind_direction_10m_cos',
    'weather_code', 'sin_halfhour', 'cos_halfhour',
    'sin_day', 'cos_day'
]

# data loading and preprocessing
df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
df = pd.concat([df1, df2], ignore_index=True)

# correct erroneous values
df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) & (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
df.loc[df["avg_available_spots"] > df["TotalSpaces"], "avg_available_spots"] = df["TotalSpaces"]

df['datetime'] = pd.to_datetime(df['datetime'])
df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
df['day_off'] = df['day_off'].astype(int)
for height in [10, 80, 120]:
    df[f'wind_direction_{height}m_sin'] = np.sin(np.deg2rad(df[f'wind_direction_{height}m']))
    df[f'wind_direction_{height}m_cos'] = np.cos(np.deg2rad(df[f'wind_direction_{height}m']))
df.drop(columns=[
    "wind_direction_10m", "wind_direction_80m", "wind_direction_120m",
    "holiday_name", "is_national_holiday"
], inplace=True)

# time split
df = df.dropna(subset=['avg_available_spots'])
train_start = pd.to_datetime("2024-01-23")
train_end = pd.to_datetime("2025-01-22")
valid_start = pd.to_datetime("2025-01-23")
valid_end = pd.to_datetime("2025-03-22")
test_start = pd.to_datetime("2025-03-23")
test_end = pd.to_datetime("2025-05-09")

df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
df_test = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

drop_cols = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID']
for dataset in [df_train, df_valid, df_test]:
    for col in drop_cols:
        if col in dataset.columns:
            dataset.drop(columns=col, inplace=True)

# feature engineering + cyclical features + One-Hot encoding
def prepare_xy_vote(df_part):
    y = df_part['avg_available_spots']
    X = df_part.drop(columns=['avg_available_spots'])

    X['sin_day'] = np.sin(2 * np.pi * X['day_val'] / 31)
    X['cos_day'] = np.cos(2 * np.pi * X['day_val'] / 31)
    X['sin_halfhour'] = np.sin(2 * np.pi * X['half_hour_interval'] / 48)
    X['cos_halfhour'] = np.cos(2 * np.pi * X['half_hour_interval'] / 48)
    X.drop(columns=['day_val', 'half_hour_interval'], inplace=True)

    cat_cols = ['month_val', 'weekday_val'] + X.select_dtypes(include=['object']).columns.tolist()
    cat_cols_exist = [col for col in cat_cols if col in X.columns]
    X = pd.get_dummies(X, columns=cat_cols_exist, drop_first=False)

    X = X.reindex(columns=selected_features_3_votes, fill_value=0)
    return X, y

X_train, y_train = prepare_xy_vote(df_train)
X_valid, y_valid = prepare_xy_vote(df_valid)
X_test, y_test = prepare_xy_vote(df_test)

# model training (using GridSearchCV)
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
model = XGBRegressor(random_state=42)
grid = GridSearchCV(model, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_

# evaluation metrics
def adjusted_r2(r2, n, k):
    return 1 - (1 - r2) * ((n - 1) / (n - k - 1))

def eval_regression(X, y, dataset_name):
    y_pred = best_model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    adj_r2 = adjusted_r2(r2, len(y), X.shape[1])
    print(f"\n📊 [{dataset_name} - 投票三票]")
    print(f"MSE: {mse:.2f}")
    print(f"Adjusted R²: {adj_r2:.4f}")

# evaluation
# eval_regression(X_train, y_train, "Train")
# eval_regression(X_valid, y_valid, "Valid")
eval_regression(X_test, y_test, "Test")


/var/folders/r9/vznk0t21541cq588wv51gt6m0000gn/T/ipykernel_3636/1861241395.py:17: DtypeWarning: Columns (47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
/var/folders/r9/vznk0t21541cq588wv51gt6m0000gn/T/ipykernel_3636/1861241395.py:18: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")


Fitting 3 folds for each of 8 candidates, totalling 24 fits

📊 [Test - 投票三票]
MSE: 128.53
Adjusted R²: 0.9654


Classification with All Features

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, precision_recall_curve
)

# data loading and preprocessing
df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
df = pd.concat([df1, df2], ignore_index=True)

df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) & (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
df.loc[df["avg_available_spots"] > df["TotalSpaces"], "avg_available_spots"] = df["TotalSpaces"]

# create classification labels: whether there is available space (>5%)
df = df.dropna(subset=['avg_available_spots', 'TotalSpaces'])
df['has_available'] = (df['avg_available_spots'] / df['TotalSpaces']) > 0.05
df['has_available'] = df['has_available'].astype(int)

# time and wind direction processing
df['datetime'] = pd.to_datetime(df['datetime'])
df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
df['day_off'] = df['day_off'].astype(int)
for height in [10, 80, 120]:
    df[f'wind_direction_{height}m_sin'] = np.sin(np.deg2rad(df[f'wind_direction_{height}m']))
    df[f'wind_direction_{height}m_cos'] = np.cos(np.deg2rad(df[f'wind_direction_{height}m']))
df.drop(columns=[
    "wind_direction_10m", "wind_direction_80m", "wind_direction_120m",
    "holiday_name", "is_national_holiday"
], inplace=True)

# time split
train_start = pd.to_datetime("2024-01-23")
train_end = pd.to_datetime("2025-01-22")
valid_start = pd.to_datetime("2025-01-23")
valid_end = pd.to_datetime("2025-03-22")
test_start = pd.to_datetime("2025-03-23")
test_end = pd.to_datetime("2025-05-09")

df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
df_test = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

drop_cols = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID', 'avg_available_spots']
for dataset in [df_train, df_valid, df_test]:
    for col in drop_cols:
        if col in dataset.columns:
            dataset.drop(columns=col, inplace=True)

# feature engineering
def prepare_xy_class(df_part):
    y = df_part['has_available']
    X = df_part.drop(columns=['has_available'])

    X['sin_day'] = np.sin(2 * np.pi * X['day_val'] / 31)
    X['cos_day'] = np.cos(2 * np.pi * X['day_val'] / 31)
    X['sin_halfhour'] = np.sin(2 * np.pi * X['half_hour_interval'] / 48)
    X['cos_halfhour'] = np.cos(2 * np.pi * X['half_hour_interval'] / 48)
    X.drop(columns=['day_val', 'half_hour_interval'], inplace=True)

    cat_cols = ['month_val', 'weekday_val'] + X.select_dtypes(include=['object']).columns.tolist()
    cat_cols_exist = [col for col in cat_cols if col in X.columns]
    X = pd.get_dummies(X, columns=cat_cols_exist, drop_first=False)

    X = X.reindex(columns=selected_features_4_votes, fill_value=0)
    return X, y

X_train, y_train = prepare_xy_class(df_train)
X_valid, y_valid = prepare_xy_class(df_valid)
X_test, y_test = prepare_xy_class(df_test)

# grid search classifier
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid = GridSearchCV(clf, param_grid=params, cv=3, scoring='f1', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
best_clf = grid.best_estimator_

# prediction probabilities and optimal threshold
y_val_prob = best_clf.predict_proba(X_valid)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_valid, y_val_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_threshold = thresholds[np.argmax(f1_scores)]

# evaluation function
def evaluate(y_true, y_prob, threshold, dataset_name):
    y_pred = (y_prob >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_prob)
    prc = average_precision_score(y_true, y_prob)
    print(f"\n📊 [{dataset_name}]")
    print(f"Threshold: {threshold:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC-AUC: {roc:.4f}")
    print(f"PRC-AUC: {prc:.4f}")

# apply the optimal threshold to the test set
y_test_prob = best_clf.predict_proba(X_test)[:, 1]
evaluate(y_test, y_test_prob, best_threshold, "Test Set")


Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:05:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:05:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:05:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:05:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr


📊 [Test Set]
Threshold: 0.2963
Accuracy: 0.9475
Precision: 0.9478
Recall: 0.9996
F1 Score: 0.9730
ROC-AUC: 0.9227
PRC-AUC: 0.9951


Elastic Net and Cost-Sensitive Learning

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)
import warnings
warnings.filterwarnings("ignore")

# preprocessing module
def load_and_clean_data():
    df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
    df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
    df = pd.concat([df1, df2], ignore_index=True)

    df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) &
           (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
    df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) &
               (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
    df.loc[df["avg_available_spots"] > df["TotalSpaces"], "avg_available_spots"] = df["TotalSpaces"]

    df['datetime'] = pd.to_datetime(df['datetime'])
    df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
    df['day_off'] = df['day_off'].astype(int)

    for h in [10, 80, 120]:
        df[f'wind_direction_{h}m_sin'] = np.sin(np.deg2rad(df[f'wind_direction_{h}m']))
        df[f'wind_direction_{h}m_cos'] = np.cos(np.deg2rad(df[f'wind_direction_{h}m']))
    df.drop(columns=[
        "wind_direction_10m", "wind_direction_80m", "wind_direction_120m",
        "holiday_name", "is_national_holiday"
    ], inplace=True)

    return df.dropna(subset=['avg_available_spots'])

# time split
def split_by_time(df):
    train_start, train_end = pd.to_datetime("2024-01-23"), pd.to_datetime("2025-01-22")
    valid_start, valid_end = pd.to_datetime("2025-01-23"), pd.to_datetime("2025-03-22")
    test_start, test_end = pd.to_datetime("2025-03-23"), pd.to_datetime("2025-05-09")

    df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
    df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
    df_test = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

    drop_cols = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID']
    for d in [df_train, df_valid, df_test]:
        for col in drop_cols:
            if col in d.columns:
                d.drop(columns=col, inplace=True)
    return df_train, df_valid, df_test

# feature processing module (for classification)
def prepare_xy_for_classification(df_part, feature_list):
    df_part['has_space'] = (df_part['avg_available_spots']/df_part['TotalSpaces'] > 0).astype(int)
    y = df_part['has_space']
    X = df_part.drop(columns=['avg_available_spots', 'has_space'])

    # cyclical features
    X['sin_day'] = np.sin(2 * np.pi * X['day_val'] / 31)
    X['cos_day'] = np.cos(2 * np.pi * X['day_val'] / 31)
    X['sin_halfhour'] = np.sin(2 * np.pi * X['half_hour_interval'] / 48)
    X['cos_halfhour'] = np.cos(2 * np.pi * X['half_hour_interval'] / 48)
    X.drop(columns=['day_val', 'half_hour_interval'], inplace=True)

    cat_cols = ['month_val', 'weekday_val'] + X.select_dtypes(include=['object']).columns.tolist()
    cat_cols_exist = [col for col in cat_cols if col in X.columns]
    X = pd.get_dummies(X, columns=cat_cols_exist, drop_first=False)

    X = X.reindex(columns=feature_list, fill_value=0)
    return X, y

# evaluation function
def evaluate_classification(model, X, y, dataset_name):
    y_prob = model.predict_proba(X)[:, 1]
    thresholds = np.linspace(0, 1, 101)
    best_f1, best_thresh = 0, 0.5

    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        f1 = f1_score(y, y_pred)
        if f1 > best_f1:
            best_f1, best_thresh = f1, t

    y_pred = (y_prob >= best_thresh).astype(int)

    print(f"\n📊 [{dataset_name} - XGBoost + Cost-sensitive]")
    print(f"Accuracy: {accuracy_score(y, y_pred):.4f}")
    print(f"Precision: {precision_score(y, y_pred):.4f}")
    print(f"Recall: {recall_score(y, y_pred):.4f}")
    print(f"F1: {f1_score(y, y_pred):.4f}")
    print(f"ROC-AUC: {roc_auc_score(y, y_prob):.4f}")
    print(f"PRC-AUC: {average_precision_score(y, y_prob):.4f}")
    print(f"Optimal Threshold: {best_thresh:.2f}")

df = load_and_clean_data()
df_train, df_valid, df_test = split_by_time(df)

# Elastic Net feature list
selected_features_elastic_net = [
"laterHourFee","firstHourFee","day_off","district",
"half_hour_cos","lat","year_val","month_val_2","TotalSpaces",
"month_val_12","lon","terrestrial_radiation_instant","dew_point_2m",
"weekday_num_2","day_sin","weekday_num_1","half_hour_sin",
"month_val_5","cloud_cover_mid","weekday_num_0","pressure_msl","vapour_pressure_deficit",
"month_val_11","wind_speed_10m",'cloud_cover','precipitation','surface_pressure']  #Elastic Net 特徵

X_train, y_train = prepare_xy_for_classification(df_train, selected_features_elastic_net)
X_valid, y_valid = prepare_xy_for_classification(df_valid, selected_features_elastic_net)
X_test, y_test = prepare_xy_for_classification(df_test, selected_features_elastic_net)

# class imbalance weights
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# training XGBoost classification model
clf = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    scale_pos_weight=scale_pos_weight,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    random_state=42
)
clf.fit(X_train, y_train)

# model evaluation
# evaluate_classification(clf, X_train, y_train, "Train")
# evaluate_classification(clf, X_valid, y_valid, "Valid")
evaluate_classification(clf, X_test, y_test, "Test")



📊 [Test - XGBoost + Cost-sensitive]
Accuracy: 0.9562
Precision: 0.9562
Recall: 1.0000
F1: 0.9776
ROC-AUC: 0.9415
PRC-AUC: 0.9971
Optimal Threshold: 0.01


Classification with Features Selected by Elastic Net

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, precision_recall_curve
)

# Elastic Net feature list
selected_features_4_votes = [
    'laterHourFee', 'firstHourFee', 'day_off', 'district', 'half_hour_cos',
    'lat', 'year_val', 'month_val_2', 'TotalSpaces', 'month_val_12', 'lon',
    'terrestrial_radiation_instant', 'dew_point_2m', 'weekday_num_2', 'day_sin',
    'weekday_num_1', 'half_hour_sin', 'month_val_5', 'cloud_cover_mid',
    'weekday_num_0', 'pressure_msl', 'vapour_pressure_deficit',
    'wind_speed_10m', 'cloud_cover', 'precipitation', 'surface_pressure'
]

# data loading and preprocessing
df1 = pd.read_csv("../區域資料_終版/中西_dataset.csv")
df2 = pd.read_csv("../區域資料_終版/安平_dataset.csv")
df = pd.concat([df1, df2], ignore_index=True)

df.loc[(df["ParkingSegmentID"] == 1192) & (df["half_hour_interval"] == 20) & (df["month_val"] == 3) & (df["day_val"] == 8), "TotalSpaces"] = 63
df.drop(df[(df['ParkingSegmentID'] == 1131) & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index, inplace=True)
df.loc[df["avg_available_spots"] > df["TotalSpaces"], "avg_available_spots"] = df["TotalSpaces"]

# create classification labels: whether there is available space (>5%)
df = df.dropna(subset=['avg_available_spots', 'TotalSpaces'])
df['has_available'] = (df['avg_available_spots'] / df['TotalSpaces']) > 0.05
df['has_available'] = df['has_available'].astype(int)

# time and wind direction processing
df['datetime'] = pd.to_datetime(df['datetime'])
df['day_off'] = df['datetime'].dt.weekday.isin([5, 6]) | df['is_national_holiday']
df['day_off'] = df['day_off'].astype(int)
for height in [10, 80, 120]:
    df[f'wind_direction_{height}m_sin'] = np.sin(np.deg2rad(df[f'wind_direction_{height}m']))
    df[f'wind_direction_{height}m_cos'] = np.cos(np.deg2rad(df[f'wind_direction_{height}m']))
df.drop(columns=[
    "wind_direction_10m", "wind_direction_80m", "wind_direction_120m",
    "holiday_name", "is_national_holiday"
], inplace=True)

# time features for half_hour_cos and half_hour_sin, day_sin
# ensure day_val and half_hour_interval are available before dropping
df['half_hour_cos'] = np.cos(2 * np.pi * df['half_hour_interval'] / 48)
df['half_hour_sin'] = np.sin(2 * np.pi * df['half_hour_interval'] / 48)
df['day_sin'] = np.sin(2 * np.pi * df['day_val'] / 31) # This is assumed to be in selected_features.
# if day_val is not in selected_features, this might not be needed.
# for now, keeping it as it's required for day_sin.

# time split
train_start = pd.to_datetime("2024-01-23")
train_end = pd.to_datetime("2025-01-22")
valid_start = pd.to_datetime("2025-01-23")
valid_end = pd.to_datetime("2025-03-22")
test_start = pd.to_datetime("2025-03-23")
test_end = pd.to_datetime("2025-05-09")

df_train = df[(df['datetime'] >= train_start) & (df['datetime'] <= train_end)].copy()
df_valid = df[(df['datetime'] >= valid_start) & (df['datetime'] <= valid_end)].copy()
df_test = df[(df['datetime'] >= test_start) & (df['datetime'] <= test_end)].copy()

# these columns should be dropped before feature selection if they are not features
drop_cols_before_feature_selection = ['datetime', 'datetime_hour', 'time', 'date', 'ParkingSegmentID', 'avg_available_spots']
for dataset in [df_train, df_valid, df_test]:
    for col in drop_cols_before_feature_selection:
        if col in dataset.columns:
            dataset.drop(columns=col, inplace=True)

# feature engineering
def prepare_xy_class(df_part):
    y = df_part['has_available']
    # Drop has_available from X
    X = df_part.drop(columns=['has_available'])

    # handle categorical columns by one-hot encoding if they are in the selected features
    # district and month_val and weekday_num related features from the list suggest one-hot encoding might be needed
    # first, identify the base categorical columns that might need one-hot encoding
    base_cat_cols = ['district', 'month_val', 'weekday_num'] # Assuming these exist as base columns before dummy creation

    # check which base categorical columns exist in the current DataFrame
    actual_base_cat_cols = [col for col in base_cat_cols if col in X.columns]

    # perform one-hot encoding
    if actual_base_cat_cols:
        X = pd.get_dummies(X, columns=actual_base_cat_cols, drop_first=False) # drop_first=False to keep all dummy variables as specified in selected_features_4_votes (e.g., month_val_2, month_val_12, weekday_num_0, weekday_num_1, weekday_num_2)

    # reindex X to only include the selected features, filling missing with 0
    # this is crucial for matching the exact feature set
    X = X.reindex(columns=selected_features_4_votes, fill_value=0)

    return X, y

X_train, y_train = prepare_xy_class(df_train)
X_valid, y_valid = prepare_xy_class(df_valid)
X_test, y_test = prepare_xy_class(df_test)

# grid search classifier
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid = GridSearchCV(clf, param_grid=params, cv=3, scoring='f1', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
best_clf = grid.best_estimator_

# prediction probabilities and optimal threshold
y_val_prob = best_clf.predict_proba(X_valid)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_valid, y_val_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_threshold = thresholds[np.argmax(f1_scores)]

# evalution function
def evaluate(y_true, y_prob, threshold, dataset_name):
    y_pred = (y_prob >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_prob)
    prc = average_precision_score(y_true, y_prob)
    print(f"\n📊 [{dataset_name}]")
    print(f"Threshold: {threshold:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC-AUC: {roc:.4f}")
    print(f"PRC-AUC: {prc:.4f}")

# apply the optimal threshold to the test set
y_test_prob = best_clf.predict_proba(X_test)[:, 1]
evaluate(y_test, y_test_prob, best_threshold, "Test Set")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [08:59:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [08:59:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [08:59:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [08:59:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr


📊 [Test Set]
Threshold: 0.4774
Accuracy: 0.9465
Precision: 0.9553
Recall: 0.9899
F1 Score: 0.9723
ROC-AUC: 0.9274
PRC-AUC: 0.9954


Classification with Features Selected by Elastic Net and Perform Oversampling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, precision_recall_curve
from xgboost import XGBClassifier 
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

# initial Elastic Net feature list
initial_elastic_net_features = [
    "laterHourFee", "firstHourFee", "day_off",
    "district", "half_hour_cos", "lat",
    "year_val", "month_val_2", "TotalSpaces",
    "month_val_12", "lon", "terrestrial_radiation_instant",
    "dew_point_2m", "weekday_num_2", "day_sin",
    "weekday_num_1", "half_hour_sin", "month_val_5",
    "cloud_cover_mid", "weekday_num_0", "pressure_msl",
    "vapour_pressure_deficit", "month_val_11", "wind_speed_10m",
    "cloud_cover", "precipitation", "surface_pressure"
]

# load data
# use smote_data_raw.csv as the source for training and validation data
df = pd.read_csv("smote_data_raw.csv")
# use df_test_out.csv as the source for test data
df_test = pd.read_csv("df_test_out.csv")

# identify the common Elastic Net selected features from both datasets
common_features = [
    col for col in initial_elastic_net_features
    if col in df.columns and col in df_test.columns
]
elastic_net_features = common_features

print(f"Using common Elastic Net features: {elastic_net_features}")
if not elastic_net_features:
    print("Warning: No common features found between initial_elastic_net_features and loaded dataframes. Check your feature list and CSV columns.")

# ensure that elastic_net_features does not include the target variable is_full
if 'is_full' in elastic_net_features:
    elastic_net_features.remove('is_full')

# correct errors and preprocess (for smote_data_raw.csv)

# use 20% as the validation set
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42) 

# process test data df_test (df_test_out.csv)
# df_test_out.csv contains the datetime column, so keep the related processing steps
if 'datetime' in df_test.columns:
    df_test['datetime'] = pd.to_datetime(df_test['datetime'])
    # Recalculate day_off because df_test_out.csv includes is_national_holiday
    if 'is_national_holiday' in df_test.columns:
        df_test['day_off'] = df_test['datetime'].dt.weekday.isin([5, 6]) | df_test['is_national_holiday']
    else:
        print("Warning: 'is_national_holiday' not found in df_test. 'day_off' will only consider weekends for df_test.")
        df_test['day_off'] = df_test['datetime'].dt.weekday.isin([5, 6])
    df_test['day_off'] = df_test['day_off'].astype(int)
else:
    print("Warning: 'datetime' not found in df_test_out.csv. 'day_off' calculation and other time-based operations might be affected for df_test.")
    if 'day_off' not in df_test.columns:
        print("Error: 'day_off' not found and cannot be calculated for df_test without 'datetime'.")


# clipping of avg_available_spots 
if 'avg_available_spots' in df_test.columns and 'TotalSpaces' in df_test.columns:
    df_test['avg_available_spots'] = df_test['avg_available_spots'].clip(upper=df_test['TotalSpaces'])
else:
    print("Warning: 'avg_available_spots' or 'TotalSpaces' not found in df_test_out.csv. Skipping clipping for df_test.")

# wind_direction_Xm_sin/cos are already in df_test_out.csv, no need to recalculate
# dynamically remove columns from df_test that are not in elastic_net_features or required for target/time processing
all_df_test_cols = df_test.columns.tolist()
cols_to_keep_for_df_test = set(elastic_net_features + ['datetime', 'avg_available_spots', 'TotalSpaces', 'is_national_holiday']) # 確保 is_national_holiday 也被保留用於 day_off 計算

columns_to_drop_df_test = [
    col for col in all_df_test_cols
    if col not in cols_to_keep_for_df_test
]
for col in columns_to_drop_df_test:
    if col in df_test.columns:
        df_test.drop(columns=[col], inplace=True)


# create classification labels (predict whether there is available parking)
# use is_full as the target variable for the training and validation sets
for data in [df_train, df_valid]:
    if 'is_full' in data.columns:
        # assuming is_full = 1 means full and is_full = 0 means available spaces
        # the target is "has space," so when is_full = 0, set has_space = 1; when is_full = 1, set has_space = 0
        data['has_space'] = (1 - data['is_full']).astype(int)
    else:
        print("Error: 'is_full' not found in smote_data_raw.csv. Cannot create 'has_space' target for train/valid.")

# the test set uses avg_available_spots and TotalSpaces to calculate has_space
if 'avg_available_spots' in df_test.columns and 'TotalSpaces' in df_test.columns:
    df_test['has_space'] = (df_test['avg_available_spots'] / df_test['TotalSpaces'].replace(0, np.nan)) > 0.05
    df_test['has_space'] = df_test['has_space'].astype(int)
else:
    print("Error: Cannot create 'has_space' target for test. 'avg_available_spots' or 'TotalSpaces' missing in df_test_out.csv.")

# features and target variable
# ensure all features exist in the dataset
# here's a check that issues a warning if some Elastic Net selected features are missing in the dataset

# check the training set features
missing_train_features = [col for col in elastic_net_features if col not in df_train.columns]
if missing_train_features:
    print(f"Error: Missing features in X_train: {missing_train_features}")
    print("Please check if 'smote_data_raw.csv' contains these columns or if preprocessing steps are correct.")

# check the test set features
missing_test_features = [col for col in elastic_net_features if col not in df_test.columns]
if missing_test_features:
    print(f"Error: Missing features in X_test: {missing_test_features}")
    print("Please check if 'df_test_out.csv' contains these columns or if preprocessing steps are correct.")


X_train = df_train[elastic_net_features]
X_valid = df_valid[elastic_net_features]
X_test = df_test[elastic_net_features]

# use the transformed has_space from is_full as the target for the training and validation sets
y_train = df_train['has_space']
y_valid = df_valid['has_space']
# use the computed has_space as the target for the test set
y_test = df_test['has_space']

# standardization + SMOTE
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

X_train_smote, y_train_smote = SMOTE(random_state=42).fit_resample(X_train_scaled, y_train)

# model training
params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}

xgb_clf = XGBClassifier(random_state=42, eval_metric='logloss', objective='binary:logistic')
grid = GridSearchCV(xgb_clf, param_grid=params, scoring='f1', cv=3, n_jobs=-1, verbose=1)
grid.fit(X_train_smote, y_train_smote)
best_model = grid.best_estimator_

# evaluation function
def evaluate_model(X, y, dataset_name):
    # check if X is empty to avoid making predictions on empty data
    if X.shape[0] == 0:
        print(f"\n📊 [{dataset_name} - Elastic Net + SMOTE + XGBoost]")
        print(f"No data to evaluate for {dataset_name}.")
        return

    proba = best_model.predict_proba(X)[:, 1]
    preds = (proba >= 0.5).astype(int)

    acc = accuracy_score(y, preds)
    prec = precision_score(y, preds)
    rec = recall_score(y, preds)
    f1 = f1_score(y, preds)
    roc_auc = roc_auc_score(y, proba)
    prc_auc = average_precision_score(y, proba)

    # search for the best threshold
    # ensure that y and proba are not empty and contain at least two classes before computing the precision_recall_curve
    if len(np.unique(y)) > 1 and len(y) > 0:
        precision, recall, thresholds = precision_recall_curve(y, proba)
        # to avoid division by zero, add a small epsilon
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
        best_idx = np.argmax(f1_scores)
        best_threshold = thresholds[best_idx]
    else:
        best_threshold = "N/A (Insufficient data for PRC)"


    print(f"\n📊 [{dataset_name} - Elastic Net + SMOTE + XGBoost]")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"PRC-AUC: {prc_auc:.4f}")
    print(f"Best Threshold (by F1): {best_threshold:.4f}")

# evaluation
# evaluate_model(X_train_scaled, y_train, "Train") 
# evaluate_model(X_valid_scaled, y_valid, "Valid") 
evaluate_model(X_test_scaled, y_test, "Test") 


Using common Elastic Net features: ['laterHourFee', 'firstHourFee', 'day_off', 'district', 'half_hour_cos', 'lat', 'year_val', 'month_val_2', 'TotalSpaces', 'month_val_12', 'lon', 'terrestrial_radiation_instant', 'dew_point_2m', 'weekday_num_2', 'day_sin', 'weekday_num_1', 'half_hour_sin', 'month_val_5', 'cloud_cover_mid', 'weekday_num_0', 'pressure_msl', 'vapour_pressure_deficit', 'month_val_11', 'wind_speed_10m', 'cloud_cover', 'precipitation', 'surface_pressure']
Fitting 3 folds for each of 8 candidates, totalling 24 fits

📊 [Test - Elastic Net + SMOTE + XGBoost]
Accuracy: 0.9072
Precision: 0.9742
Recall: 0.9265
F1-score: 0.9498
ROC-AUC: 0.9015
PRC-AUC: 0.9937
Best Threshold (by F1): 0.0546
